In [1]:
import jax
import numpy as np
import wandb
import matplotlib.pyplot as plt
from collections import defaultdict

from config import config
from tensorneat.algorithm.hyperneat import FullSubstrate, MLPSubstrate, DefaultSubstrate
from substrate_generation.pca_coor_generator import PCAanalyzer 
from substrate_generation.pca_inv_coor_generator import InvPCAanalyzer 
from substrate_generation.dl_coor_generator import DictionaryAnalyzer
from substrate_generation.fa_coor_generator import FactorAnalyzer
from substrate_generation.manual_coor_generator import ManualIOMapper
from substrate_generation.random_coor_generator import RandomCoordinateGenerator
from substrate_generation.data_sampling import collect_random_policy_data, collect_trained_agent_policy_data
from evol_pipeline.brax_env import CustomBraxEnv
from evol_pipeline.custom_pipeline import CustomPipeline
from evol_pipeline.custom_substrate import AutoLayeredCoordMLPSubstrate
from substrate_generation.hidden_layers import HiddenLayerGenerator
from utils.visualization import visualize_cppn, visualize_nn, display_plots_side_by_side
from utils.utils import save_coordinates_to_csv, setup_folder_structure
from evol_pipeline.evol_algorithm import create_evol_algorithm

In [2]:
OUTPUT_DIR = config["experiment"]["output_dir"]
setup_folder_structure(OUTPUT_DIR)

In [3]:
env_name = config["experiment"]["env_name"]
env_problem = CustomBraxEnv(
    env_name=env_name,
    backend=config["environment"]["backend"],
    brax_args=config["environment"]["brax_args"],
    max_step=config["environment"]["max_step"],
    repeat_times=config["environment"]["repeat_times"],
    obs_normalization=config["environment"]["obs_normalization"],
    sample_episodes=config["environment"]["sample_episodes"],
)
obs_size = env_problem.input_shape[0]
act_size = env_problem.output_shape[0]
feature_dims = config["data_analysis"]["feature_dims"][1]

print("env_problem.input_shape: ", env_problem.input_shape)
print("env_problem.input_shape: ", env_problem.output_shape)

/home/andi/anaconda3/envs/jax/lib/python3.10/site-packages/brax/io/mjcf.py:480: UserWarning: Brax System, piplines and environments are not actively being maintained. Please see MJX for a well maintained JAX-based physics engine: https://github.com/google-deepmind/mujoco/tree/main/mjx. For a host of environments that use MJX, see: https://github.com/google-deepmind/mujoco_playground.
  warnings.warn(
2025-10-29 17:16:12.278918: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-10-29 17:16:23.410303: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by usin

env_problem.input_shape:  (27,)
env_problem.input_shape:  (8,)


In [4]:
key = jax.random.PRNGKey(config["experiment"]["seed"]) # Use seed from config
key, random_key = jax.random.split(key)
num_random_sampling_steps = 5000
random_data = collect_random_policy_data(
    env_problem, random_key, 
    num_random_sampling_steps, 
    obs_diff_only=True,
    do_normalization=False,
)

Starting data collection for 5000 steps using a random policy...


2025-10-29 17:16:29.968104: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.
2025-10-29 17:16:29.968120: W external/xla/xla/service/gpu/autotuning/dot_search_space.cc:200] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs?Working around this by using the full hints set instead.


Causal data collection finished.


In [5]:
variance = config["data_analysis"]["variance_threshold"]

print(f"Analyzing random sampling data")
analyzer_pca = PCAanalyzer(
    data=random_data, 
    obs_size=obs_size, 
    act_size=act_size,
    variance_threshold=variance, 
    feature_dims=feature_dims,
    width_factor=config["substrate"]["width_factor"],
    normalize_coors=config["data_analysis"]["normalize_coors"],
    depth_factor=config["substrate"]["depth_factor"],
)
input_coors, output_coors = analyzer_pca.generate_io_coordinates()
analyzer_pca.plot_variance(save_path=f"{OUTPUT_DIR}/data_analysis/pca_variance_random.png")
analyzer_pca.plot_principal_components(save_path=f"{OUTPUT_DIR}/data_analysis/pca_heatmap_random.png")
print("\n")

Analyzing random sampling data
Running PCA to find feature dimensions covering 100.0% of variance (with a hard limit of 2 dimensions)...
PCA found 2 dimensions needed for 100.0% variance.
Applying max limit. Final number of feature dimensions: 2
Normalizing coordinates...
Added layering dimension. Final coordinate size: 3
PCA variance plot saved to: output/ant/data_analysis/pca_variance_random.png
Principal component heatmap saved to: output/ant/data_analysis/pca_heatmap_random.png




In [6]:
hidden_layer_gen = HiddenLayerGenerator(
    env_name=env_name,
    obs_size=obs_size,
    act_size=act_size,
    hidden_layer_type=config["substrate"]["hidden_layer_type"],
    hidden_depth=config["substrate"]["hidden_depth"],
    depth_factor=config["substrate"]["depth_factor"],
    width_factor=config["substrate"]["width_factor"],
)

substrates = defaultdict(lambda: defaultdict(dict))

In [7]:
hidden_coors = hidden_layer_gen.get_hidden_coors(input_coors=input_coors)

active_substrate = FullSubstrate(
    input_coors=input_coors,
    hidden_coors=hidden_coors,
    output_coors=output_coors
)


In [8]:

evol_algorithm = create_evol_algorithm(substrate=active_substrate)

initial_cppn_layers = config["algorithm"]["genome"]["cppn_init_hidden_layers"](active_substrate.query_coors.shape[1])
print("Intial CPPN Layers:", initial_cppn_layers)

substrate_dimensions = int(active_substrate.query_coors.shape[1]/2)

wanbd_name = f"{env_name}_random_pca_test_{substrate_dimensions}d"
wandb_tags = [config["substrate"]["hidden_layer_type"], env_name, "pca", "random", "test", f"{config['substrate']['hidden_depth']}_hl", f"{config['algorithm']['neat']['pop_size']}pop", f"{config['environment']['backend']}"]

wandb.init(
    name=wanbd_name,
    project="substrate_gen",
    tags=wandb_tags,
    config=config  
)

wandb.config.update(
    {
        "substrate": {
            "obs_size": obs_size,
            "act_size": act_size,
            "num_queries": active_substrate.query_coors.shape[0],
            "query_dim": active_substrate.query_coors.shape[1],
            },
        "algorithm": {
            "neat": {
                "num_inputs": evol_algorithm.num_inputs,
                },
            "genome": {
                "cppn_init_hidden_layers": initial_cppn_layers,
                },
            },
    },
)


pipeline = CustomPipeline(
    algorithm=evol_algorithm,
    problem=env_problem,
    seed=config["experiment"]["seed"],
    generation_limit=config["pipeline"]["generation_limit"],
    fitness_target=config["pipeline"]["fitness_target"],
    is_save=True,
    save_dir=config["experiment"]["output_dir"],
)

init_state = pipeline.setup()
state = pipeline.auto_run(state=init_state)

print(f"\nTraining finished. Best fitness achieved: {pipeline.best_fitness}")

wandb.finish()

state_for_show = state[0] if isinstance(state, tuple) else state

# Transform the best genome into network parameters
best_genome = pipeline.best_genome

# Built-in show method to produce and save a video of the agent
pipeline.show(
    state=state_for_show,
    best=best_genome,
    output_type="mp4",
    save_path=f"{OUTPUT_DIR}/video/agent_random_pca_test.mp4",
)

# Visualizes the CPPN
visualize_cppn(
    pipeline=pipeline, 
    state=state, 
    save_path=f"{OUTPUT_DIR}/topology/cppn_random_pca_test.svg"
    )
# Visualizes a representation of the neural network in 2D space
visualize_nn(
    pipeline=pipeline, 
    state=state, 
    save_path=f"{OUTPUT_DIR}/topology/nn_random_pca_test_.svg", 
    substrate=active_substrate, 
    input_coors=input_coors, 
    hidden_coors=hidden_coors, 
    output_coors=output_coors, 
    hidden_depth=config["substrate"]["hidden_depth"], 
    max_weight=config["algorithm"]["hyperneat"]["max_weight"], 
    )

# all input and output coordinates are logged for further analysis
log_coors = input_coors
for coor in hidden_coors:
    log_coors.append(coor)
for coor in output_coors:
    log_coors.append(coor)
save_coordinates_to_csv(
    coordinates=log_coors,
    filepath=f"{OUTPUT_DIR}/coordinates/random_pca_test_io.csv",
)


Intial CPPN Layers: [2]


wandb: Currently logged in as: wirkelzirkel (wirkelzirkel-iu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


save to output/ant
initializing
initializing finished
start compile
compile finished, cost time: 33.981213s
Generation: 1, Cost time: 26623.24ms
 	fitness: valid cnt: 3500, max: 54.0870, min: -3962.8181, mean: -1693.6022, std: 1294.7893

	node counts: max: 10, min: 8, mean: 9.27
 	conn counts: max: 16, min: 6, mean: 14.27
 	species: 20, [79, 62, 67, 115, 1, 1, 1, 18, 41, 50, 28, 30, 1, 2, 1, 64, 36, 23, 45, 2835]

Generation: 2, Cost time: 26433.30ms
 	fitness: valid cnt: 3500, max: 54.8435, min: -3975.2114, mean: -670.8160, std: 1121.6263

	node counts: max: 11, min: 8, mean: 9.43
 	conn counts: max: 18, min: 6, mean: 14.52
 	species: 20, [202, 22, 187, 211, 32, 207, 24, 15, 124, 129, 93, 102, 98, 38, 59, 41, 32, 24, 16, 1844]

Generation: 3, Cost time: 26147.54ms
 	fitness: valid cnt: 3500, max: 55.3055, min: -3956.4009, mean: -301.7120, std: 845.0309

	node counts: max: 12, min: 8, mean: 9.68
 	conn counts: max: 20, min: 1, mean: 14.92
 	species: 20, [266, 203, 2, 231, 89, 224, 130,

KeyboardInterrupt: 

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x74a3683bf250>> (for post_run_cell), with arguments args (<ExecutionResult object at 74a751f34760, execution_count=8 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 74a751f34850, raw_cell="
evol_algorithm = create_evol_algorithm(substrate=.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/home/andi/Dokumente/Bachelorarbeit/dim_tuning/tests.ipynb#X10sZmlsZQ%3D%3D> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe